## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from numpy import defaultdict

## Grid World Environment Setting

In [ ]:
# parameter of environment setting
HEIGHT = 6
WIDTH = 6
TRANSITION_PROB = 1
POSSIBLE_ACTIONS = [0,1,2,3] # left, right, up, down
ACTIONS =[(-1,0), (1,0), (0,-1), (0,1)] # actions represented by grid points.
REWARDS = []

In [5]:
class Env:
    def __init__(self):
        self.transition_probability = TRANSITION_PROB
        self.width = WIDTH
        self.height = HEIGHT
        self.reward = [[0] * WIDTH for _ in range(HEIGHT)]
        self.possible_actions = POSSIBLE_ACTIONS
        self.reward[6][6] = 3 # desirable destination
        self.reward[1][4] = -1 # 
        self.reward[2][1] = -1 
        self.reward[2][5] = -1 
        self.reward[5][4] = -1
        self.reward[6][1] = -1
        self.all_state = []

        for x in range(WIDTH):
            for y in range(HEIGHT):
                state = [x, y]
                self.all_state.append(state)
    
    def get_reward(self, state, action):
        next_state = self.state_after_action(state, action)
        return self.reward[next_state[0]][next_state[1]]

    def state_after_action(self, state, action_index):
        action = ACTIONS[action_index]
        return self.check_boundary([state[0] + action[0], state[1] + action[1]])

    # static method
    @staticmethod
    def check_boundary(state):
        state[0] = (0 if state[0] < 0 else WIDTH - 1 if state[0] > WIDTH - 1 else state[0])
        state[1] = (0 if state[1] < 0 else HEIGHT - 1 if state[1] > HEIGHT - 1 else state[1])
        return state

    def get_transition_prob(self, state, action):
        return self.transition_probability

    def get_all_states(self):
        return self.all_state



## Q - Learning Declaration

In [ ]:
def __init__(self, actions):
    self.actions = actions
    self.step_size = 0.01
    self.discount_factor = 0.9
    self.epsilon = 0.9
    self.q_table = defaultdict(lambda: [0.0, 0.0, 0.0, 0.0])

def learn(self, state, action, reward, next_state):
    state, next_stae = str(state), str(next_state)
    q_1 = self.q_table[state][action]
    q_2 = reward + self.discount_factor * max(self.q_table[next_state])
    self.q_table[state][action] += self.step_size * (q_2 - q_1)

def get_action(self, state):
    if np.random.rand() < self.epsilon:
        action = np.random.choice(self.actions)
    else:
        state = str(state)
        q_list = self.q_table[state]
        action = np.arg_max(q_list)
    return action

In [ ]:
env = Env()
agent = QLearningAgent(actions = list(range(env.n_actions)))

for episode in range(1000):
    env.render()

    action = agent.get_action(state)

    next_state, reward, done = env.step(action)

    agent.learn(state, action, reward, next_state)

    state = next_state

    env.print_value_all(agent.q_table)    